In [1]:
import pandas as pd

#### Загрузим тестовые данные

In [2]:
start_data = pd.read_csv('input/train.csv')

#### Обработаем тестовые данные

#### Больше всего незаполненных данных в поле "Healthcare_1". Заполним эти поля в зависимости от поля "Social_3". Путь нахождения метода заполнения см. в файле "Поиск замены NaN для Healthcare_1"

In [3]:
def fill_Healthcare_1_by_Social_3(df, result_df):
    all_means = df['Healthcare_1'].mean()
    means_res = result_df.groupby('Social_3')[['Healthcare_1']].mean()
    NaN_means = means_res.loc[means_res['Healthcare_1'].isnull()]
    NoNaN_means = means_res.loc[means_res['Healthcare_1'].notnull()]
    NaN_means_s = NaN_means.index.values
    NoNaN_means_s = NoNaN_means.index.values
    result_df.loc[result_df['Social_3'].isin(NaN_means_s), 'Healthcare_1'] = all_means
    for i in NoNaN_means_s:
       result_df.loc[(result_df['Social_3'] == i) & 
                  (result_df['Healthcare_1'].isnull()), 'Healthcare_1'] = means_res.loc[i, 'Healthcare_1']

#### Обработаем остальные данные.
#### Вначале заменим квартиры с нулевым количеством комнат на квартиры с одной комнатой.
#### После этого обработаем площадь. Максимальную трогать не будем, подразумевая, что это может быть квартира со свободной планировкой. Минимально возможную примем как 10 + 5 м. на каждую комнату
#### Жилую площадь, где она незаполнена, принимаем как 90% от общей.
#### Этажность дома, если она равна нулю или меньше этажа квартиры, примем как этаж квартиры + 2 этажа.
#### Если есть год дома больше 2020 или меньше 1800 присвоим этим домам среднее за вычетом подобных значений.

In [4]:
def clear_other(df):
    df.loc[df['Rooms'] == 0] = 1
    df.loc[df['Square'] < 10 + 5 * df['Rooms'], 'Square'] = 10 + 5 * df['Rooms']

    df.loc[df['LifeSquare'].isnull(), 'LifeSquare'] = df['Square'] * 0.9

    df.loc[
        ((df['HouseFloor'] == 0) | (df['HouseFloor'] < df['Floor'])), 'HouseFloor'
        ] = df['Floor'] + 2

    mean_year = df.loc[
        ((df['HouseYear'] < 2020) & (df['HouseYear'] > 1800)), 'HouseYear'
        ].mean()
    df.loc[
        ((df['HouseYear'] > 2020) | (df['HouseYear'] < 1800)), 'HouseYear'
        ] = mean_year

#### И последняя обраболтка - есть три поля, содержащие A и B. Заменим их значения на числовые значения.

In [5]:
def clear_AB(df):
    df['Shops_2'] = (df['Shops_2'] == 'A').astype(int)
    df['Ecology_2'] = (df['Ecology_2'] == 'A').astype(int)
    df['Ecology_3'] = (df['Ecology_3'] == 'A').astype(int)

#### Очищаем тестовые данные

In [6]:
fill_Healthcare_1_by_Social_3(start_data, start_data)

In [7]:
clear_other(start_data)

In [8]:
clear_AB(start_data)

#### Разбиваем на параметры и исследуемую величину

In [9]:
X = start_data.loc[:, :'Shops_2']
y = pd.DataFrame(start_data.loc[:, 'Price'])

#### Создаём модель обучения

In [10]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
model =  RandomForestRegressor(n_estimators=100, max_depth=7, random_state=42)

#### Делим данные на тренировочные и проверочные

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

#### Производим обучение

In [14]:
model.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

#### И по обученной моджели предсказываем исследуемую величину в проверочных данных

In [15]:
y_pred = model.predict(X_valid)

#### проверим получившийся коэффициент R2

In [16]:
from sklearn.metrics import r2_score as r2

In [17]:
r2(y_valid, y_pred)

0.6702991344741639

#### Получилось 0.67, что нас вполне устроит.

#### Загружаем тестовые данные

In [18]:
test_data = pd.read_csv('input/test.csv')

#### Очищаем их

In [19]:
fill_Healthcare_1_by_Social_3(start_data, test_data)

In [20]:
clear_other(test_data)

In [21]:
clear_AB(test_data)

#### Проверяем, что все данные заполнены и все в числовых форматах

In [22]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null float64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null int32
Ecology_3        5000 non-null int32
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     5000 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null int32
dtypes: float64(8), int32(3), int64(8)
memory usage: 683.7 KB


#### Предсказываем исследуемую величину для тестовых данных, создаём необходимый для вывода датафрейм и сохраняем его в файл

In [23]:
y_pred_2 = model.predict(test_data)

In [24]:
res_data = pd.DataFrame(y_pred_2, columns=['Price'])
res_data['Id'] = test_data['Id']

In [25]:
res_data.loc[:, ['Id', 'Price']].to_csv('AHidden_predictions.csv', index=False)